# Devolver fuentes

A menudo, en aplicaciones de preguntas y respuestas, es importante mostrar a los usuarios las fuentes que se utilizaron para generar la respuesta. La forma más sencilla de hacer esto es que la cadena devuelva los Documentos que se recuperaron en cada generación.

Trabajaremos con la aplicación de preguntas y respuestas que construimos sobre la publicación del blog "LLM Powered Autonomous Agents" de Lilian Weng en el inicio rápido.

# Configuración

## Dependencias
Utilizaremos un modelo de chat de OpenAI, incrustaciones y un vector store Chroma en este recorrido, pero todo lo mostrado aquí funciona con cualquier ChatModel o LLM, Embeddings y VectorStore o Retriever.

Usaremos los siguientes paquetes:

In [1]:
# %pip install --upgrade --quiet langchain langchain-community langchainhub langchain-openai chromadb bs4

Note: you may need to restart the kernel to use updated packages.


Necesitamos establecer la variable de entorno OPENAI_API_KEY, lo cual se puede hacer directamente o cargar desde un archivo .env de la siguiente manera:

In [1]:
#import getpass
#import os

#os.environ["OPENAI_API_KEY"] = getpass.getpass()

import dotenv

dotenv.load_dotenv()

True

## LangSmith

Muchas de las aplicaciones que construyas con LangChain contendrán múltiples pasos con múltiples invocaciones de llamadas a LLM. A medida que estas aplicaciones se vuelven más complejas, se vuelve crucial poder inspeccionar qué está sucediendo exactamente dentro de tu cadena o agente. La mejor manera de hacerlo es con LangSmith.

Ten en cuenta que LangSmith no es necesario, pero es útil. Si deseas utilizar LangSmith, después de registrarte en el enlace anterior, asegúrate de establecer tus variables de entorno para comenzar a registrar trazas:

In [2]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# Cadena sin fuentes
Aquí está la aplicación de preguntas y respuestas que construimos sobre la publicación del blog "Agentes Autónomos Potenciados por LLM" de Lilian Weng en el Inicio Rápido:

In [3]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [4]:
# Load, chunk and index the contents of the blog.
bs_strainer = bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs_strainer},
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [5]:
rag_chain.invoke("¿Que es la descompocision de tareas?")

'La descomposición de tareas es el proceso de dividir una tarea compleja en pasos más pequeños y manejables. Esto permite a un agente abordar la tarea de manera más eficiente y comprender mejor el proceso de pensamiento necesario para completarla. La descomposición de tareas puede realizarse mediante técnicas como Chain of Thought (CoT) o Tree of Thoughts.'


# Añadiendo fuentes
Con LCEL, es fácil devolver los documentos recuperados:

In [7]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

rag_chain_with_source.invoke("¿Que es la descompocision de tareas?")

{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search p